Agenda:
1. [Get data](#Get-the-data)

3. [Tidy](#Tidy-it-up)


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ftplib import FTP
import datetime
import re
import zipfile
import os

%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 5) # set default size of plots

sns.set_style("white")
sns.set_context("talk")
sns.set_palette('Set2', 10)

## Get the data
There are two options - TransitFeeds and the workshop's S3 bucket.

In [57]:
!aws s3 ls s3://s3.obus.hasadna.org.il/old_gtfs/

                           PRE ./
2017-07-20 23:47:58  261938715 2015_09_11_06_43_15.zip
2017-07-21 00:00:21  236739837 2015_09_13_13_38_19.zip
2017-07-21 00:11:49  235367151 2015_09_14_10_56_41.zip
2017-07-21 00:22:47   91461331 2015_11_03_13_05_55.zip
2017-07-21 00:27:07  130548315 2015_12_07_18_10_02.zip
2017-07-21 00:33:15  130548315 2015_12_07_18_14_01.zip
2017-07-21 00:39:23  130548315 2015_12_07_18_24_47.zip
2017-07-21 00:45:27  123755272 2015_12_08_01_00_03.zip
2017-07-21 00:51:18  110770319 2015_12_09_01_00_03.zip
2017-07-21 00:56:33  110408652 2015_12_10_01_00_02.zip
2017-07-21 01:01:46  110127193 2015_12_11_01_00_03.zip
2017-07-21 01:07:01  108863696 2015_12_12_01_00_05.zip
2017-07-21 01:12:10  105081885 2015_12_13_01_00_03.zip
2017-07-21 01:17:06  103712539 2015_12_14_01_00_03.zip
2017-07-21 01:22:03   98530870 2015_12_15_01_00_03.zip
2017-07-21 01:26:52   91589519 2015_12_16_01_00_03.zip
2017-07-21 01:31:14   92013076 2015_12_17_01_00_04.zip
2017-07-21 01:35:36   92533075 

In [58]:
!aws s3 cp s3://s3.obus.hasadna.org.il/old_gtfs/2016_05_06_00_00_02.zip data/gtfs_feeds/2016-05-06.zip


Completed 256.0 KiB/153.9 MiB (93.8 KiB/s) with 1 file(s) remaining
Completed 512.0 KiB/153.9 MiB (186.5 KiB/s) with 1 file(s) remaining
Completed 768.0 KiB/153.9 MiB (278.2 KiB/s) with 1 file(s) remaining
Completed 1.0 MiB/153.9 MiB (368.9 KiB/s) with 1 file(s) remaining  
Completed 1.2 MiB/153.9 MiB (458.1 KiB/s) with 1 file(s) remaining  
Completed 1.5 MiB/153.9 MiB (534.1 KiB/s) with 1 file(s) remaining  
Completed 1.8 MiB/153.9 MiB (618.4 KiB/s) with 1 file(s) remaining  
Completed 2.0 MiB/153.9 MiB (702.8 KiB/s) with 1 file(s) remaining  
Completed 2.2 MiB/153.9 MiB (786.3 KiB/s) with 1 file(s) remaining  
Completed 2.5 MiB/153.9 MiB (869.3 KiB/s) with 1 file(s) remaining  
Completed 2.8 MiB/153.9 MiB (929.7 KiB/s) with 1 file(s) remaining  
Completed 3.0 MiB/153.9 MiB (1008.9 KiB/s) with 1 file(s) remaining 
Completed 3.2 MiB/153.9 MiB (1.0 MiB/s) with 1 file(s) remaining    
Completed 3.5 MiB/153.9 MiB (1.1 MiB/s) with 1 file(s) remaining    
Completed 3.8 MiB/153.9 MiB (1.2 Mi

In [24]:
!aws s3 cp s3://s3.obus.hasadna.org.il/2017-01-01.zip data/gtfs_feeds/2017-01-01.zip


Completed 256.0 KiB/126.6 MiB (99.0 KiB/s) with 1 file(s) remaining
Completed 512.0 KiB/126.6 MiB (195.2 KiB/s) with 1 file(s) remaining
Completed 768.0 KiB/126.6 MiB (280.4 KiB/s) with 1 file(s) remaining
Completed 1.0 MiB/126.6 MiB (362.6 KiB/s) with 1 file(s) remaining  
Completed 1.2 MiB/126.6 MiB (448.3 KiB/s) with 1 file(s) remaining  
Completed 1.5 MiB/126.6 MiB (538.0 KiB/s) with 1 file(s) remaining  
Completed 1.8 MiB/126.6 MiB (627.7 KiB/s) with 1 file(s) remaining  
Completed 2.0 MiB/126.6 MiB (709.4 KiB/s) with 1 file(s) remaining  
Completed 2.2 MiB/126.6 MiB (697.8 KiB/s) with 1 file(s) remaining  
Completed 2.5 MiB/126.6 MiB (755.8 KiB/s) with 1 file(s) remaining  
Completed 2.8 MiB/126.6 MiB (811.1 KiB/s) with 1 file(s) remaining  
Completed 3.0 MiB/126.6 MiB (779.7 KiB/s) with 1 file(s) remaining  
Completed 3.2 MiB/126.6 MiB (803.7 KiB/s) with 1 file(s) remaining  
Completed 3.5 MiB/126.6 MiB (865.5 KiB/s) with 1 file(s) remaining  
Completed 3.8 MiB/126.6 MiB (920.4 

## Tidy it up
Again I'm using [partridge](https://github.com/remix/partridge/tree/master/partridge) for filtering on dates, and then some tidying up and transformations.

In [7]:
from gtfs_utils import *

TARIFF_FILE_NAME = 'Tariff.zip'
TARIFF_TXT_NAME = 'Tariff.txt'
TARIFF_TO_REFORM_ZONE = 'StationToReformZone.txt'
local_tariff_path = 'data/sample/tariff.zip' 

In [ ]:
#conn = ftp_connect()
#get_ftp_file(conn, file_name = TARIFF_FILE_NAME, local_zip_path = local_tariff_path )

In [8]:
# not a true csv, so we need to jiggle it a bit
cols = ['ShareCode','ShareCodeDesc','ZoneCodes','Daily','Weekly','Monthly','FromDate','ToDate', 'EXTRA']
reform_cols = ['StationId', 'ReformZoneCode','FromDate','ToDate', 'EXTRA']
with zipfile.ZipFile(local_tariff_path) as zf:
    tariff_df = (pd.read_csv(zf.open(TARIFF_TXT_NAME), header=None, skiprows=[0], names = cols)
                .drop(columns = ['EXTRA']))
    reform_df = (pd.read_csv(zf.open(TARIFF_TO_REFORM_ZONE), header=None, skiprows=[0], names = reform_cols)
                 .drop(columns = ['EXTRA']))

    
# remove ShareCodes which contain multiple zones  e.g. גוש דן מורחב
tariff_df = (tariff_df[~ tariff_df.ZoneCodes.str.contains(';')]
             .rename(columns = {'ShareCodeDesc': 'zone_name',
                               'ZoneCodes': 'zone_id'}))
rs = reform_df[['StationId', 'ReformZoneCode']].drop_duplicates().applymap(str).set_index('StationId').iloc[:,0]

ts = (tariff_df[['zone_id', 'zone_name']].drop_duplicates().set_index('zone_id').iloc[:,0])
zones = rs.map(ts).reset_index().rename(columns={'StationId': 'stop_code', 'ReformZoneCode':'zone_name'})

In [10]:
import partridge as ptg

def get_partridge_feed(zip_path, date):
    service_ids_by_date = ptg.read_service_ids_by_date(zip_path)
    service_ids = service_ids_by_date[date]

    feed = ptg.feed(zip_path, view={
        'trips.txt': {
            'service_id': service_ids,
        },
    })
    return feed
    
def to_timedelta(df):
    '''
    Turn time columns into timedelta dtype
    '''
    cols = ['arrival_time', 'departure_time']
    numeric = df[cols].apply(pd.to_timedelta, unit='s')
    df = df.copy()
    df[cols] = numeric
    return df

def get_tidy_feed_df(feed, zones):
    s = feed.stops
    r = feed.routes
    t = (feed.trips
         .assign(route_id=lambda x: pd.Categorical(x['route_id'])))
    f = (feed.stop_times[['trip_id', 'departure_time', 'arrival_time', 'stop_id', 'stop_sequence']]
         .merge(s[['stop_id', 'stop_name', 'stop_lat', 'stop_lon', 'stop_code']], on='stop_id')
         # Much faster joins and slices with Categorical dtypes
         .merge(zones, how='left')
         .assign(zone_name=lambda x: pd.Categorical(x['zone_name']))
         .merge(t[['trip_id', 'route_id', 'direction_id']], on='trip_id')
         .merge(r[['route_id', 'route_short_name', 'route_long_name']], on='route_id')
         .assign(route_id=lambda x: pd.Categorical(x['route_id']))
         .pipe(to_timedelta)
        )
    return f

In [25]:
LOCAL_ZIP_PATH = 'data/gtfs_feeds/2017-01-01.zip' 

In [26]:
feed = get_partridge_feed(LOCAL_ZIP_PATH, datetime.date(2017,1 , 1))

In [14]:
f = get_tidy_feed_df(feed, zones)

In [43]:
with zipfile.ZipFile(LOCAL_ZIP_PATH) as zf:
    stops = pd.read_csv(zf.open('stops.txt'))
    
stops[stops.stop_code==40265]

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,location_type,parent_station,zone_id
965,1187,40200,הגיבורים/העלייה הראשונה,רחוב:הגיבורים 68 עיר: חדרה רציף: קומה:,32.436182,34.922808,0,NaN,702.0


In [55]:
stops[(stops.stop_name.str.contains('חבצלת')) & (stops.stop_desc.str.contains('בנימינה'))]

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,location_type,parent_station,zone_id
1213,1538,40625,רחוב חי/סמטת החבצלת,רחוב:חי 70 עיר: בנימינה גבעת עדה רציף: קומה:,32.517556,35.007224,0,NaN,702.0
24802,36072,47872,חבצלת/הארז,רחוב:חבצלת 4 עיר: בנימינה גבעת עדה רציף: קו...,32.527087,34.939577,0,NaN,702.0
24805,36075,47875,חבצלת/הארז,רחוב:חבצלת 3 עיר: בנימינה גבעת עדה רציף: קו...,32.526611,34.939991,0,NaN,702.0


In [50]:
stops[stops.stop_code>40250].sort_values(by='stop_code')

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,location_type,parent_station,zone_id
996,1227,40251,צומת בנימינה,רחוב:653 עיר: בנימינה גבעת עדה רציף: קומה:,32.517476,34.930838,0,NaN,702.0
997,1228,40252,משרדי חברת חשמל/צה''ל,רחוב:צה''ל 51 עיר: חדרה רציף: קומה:,32.440961,34.931859,0,NaN,702.0
998,1229,40253,צומת מעגן מיכאל,רחוב:4 עיר: חוף הכרמל רציף: קומה:,32.556372,34.931537,0,NaN,702.0
999,1230,40254,צומת מעגן מיכאל,רחוב:4 עיר: חוף הכרמל רציף: קומה:,32.557378,34.932279,0,NaN,702.0
1000,1231,40255,משרדי חברת חשמל/צה''ל,רחוב:צה''ל 50 עיר: חדרה רציף: קומה:,32.441624,34.932884,0,NaN,702.0
1001,1233,40257,זיסו/יפו,רחוב:מסילת הברזל עיר: חיפה רציף: קומה:,32.827351,34.987691,0,NaN,310.0
1002,1234,40258,דרך העצמאות/הנדיב,רחוב:דרך העצמאות 54 עיר: בנימינה גבעת עדה רצי...,32.523856,34.934897,0,NaN,702.0
1003,1236,40260,דרך העצמאות/נורית,רחוב:דרך העצמאות 82 עיר: בנימינה גבעת עדה רצי...,32.527124,34.935504,0,NaN,702.0
1004,1238,40262,חטיבת הנחל/נחל צין,רחוב:חטיבת הנח''ל 37 עיר: חדרה רציף: קומה:,32.431152,34.936468,0,NaN,702.0
1005,1243,40267,משטרה/שדרות ניל''י,רחוב:שדרות ניל''י עיר: זכרון יעקב רציף: קו...,32.571946,34.938372,0,NaN,334.0


In [ ]:
LZ = 